<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_Part3_Chapter04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#상관관계

In [1]:
import pandas as pd

data = {
    '키' : [150, 160, 170, 175, 165],
    '몸무게' : [42, 50, 70, 64, 56]
}
df = pd.DataFrame(data)

In [2]:
#상관계수 구하는 법 1
display(df.corr())
print(df.corr().iloc[0,1])

,키,몸무게
키,1.000000,0.919509
몸무게,0.919509,1.000000


0.9195090879163764


In [3]:
#상관계수 구하는 법 2
print(df['키'].corr(df['몸무게']))
print(df['몸무게'].corr(df['키']))

0.9195090879163765
0.9195090879163765


In [4]:
#상관계수 구하는 방법 바꾸기
display(df.corr(method='pearson'))
display(df.corr(method='spearman'))
display(df.corr(method='kendall'))

,키,몸무게
키,1.000000,0.919509
몸무게,0.919509,1.000000


,키,몸무게
키,1.0,0.9
몸무게,0.9,1.0


,키,몸무게
키,1.0,0.8
몸무게,0.8,1.0


In [5]:
from scipy import stats

#피어슨 상관계수와 p-value
print(stats.pearsonr(df['몸무게'], df['키']))
#스피어만 상관계수와 p-value
print(stats.spearmanr(df['몸무게'], df['키']))
#켄달의 타우 상관계수와 p-value
print(stats.kendalltau(df['몸무게'], df['키']))


PearsonRResult(statistic=np.float64(0.9195090879163766), pvalue=np.float64(0.02707945689558947))
SignificanceResult(statistic=np.float64(0.8999999999999998), pvalue=np.float64(0.03738607346849874))
SignificanceResult(statistic=np.float64(0.7999999999999999), pvalue=np.float64(0.08333333333333333))


#단순 선형 회귀 분석

In [6]:
import pandas as pd

data = {
    '키' : [150, 160, 170, 175, 165, 155, 172, 168, 174, 158,
           162, 173, 156, 159, 167, 163, 171, 169, 176, 161],
    '몸무게' : [42, 50, 70, 64, 56, 48, 68, 60, 65, 52,
                54, 67, 49, 51, 58, 55,69, 61, 66, 53]
}
df = pd.DataFrame(data)

In [7]:
from statsmodels.formula.api import ols
model = ols('키~몸무게', data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      키   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     148.0
Date:                Sat, 11 Oct 2025   Prob (F-statistic):           4.04e-10
Time:                        02:07:05   Log-Likelihood:                -45.761
No. Observations:                  20   AIC:                             95.52
Df Residuals:                      18   BIC:                             97.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    115.0676      4.158     27.671      0.000     106.331     123.804
몸무게            0.8658      0.071     12.167      0.000       0.716       1.015
==============================================================================
Omnibus:                        0.985   Durbin-Watson:                   2.609
Prob(Omnibus):                  0.611   Jarque-Bera (JB):                0.336
Skew:                          -0.315   Prob(JB):                        0.845
Kurtosis:                       3.082   Cond. No.                         432.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
print('r2score :', model.rsquared)
print('몸무게 기울기 : ', model.params['몸무게'])
print('절편 : ', model.params['Intercept'])
print("pvalue : {:.10f}".format(model.pvalues['몸무게']))

r2score : 0.8915914350087264
몸무게 기울기 :  0.8658438852380201
절편 :  115.06763904471865
pvalue : 0.0000000004


In [10]:
#새로운 데이터 예측
new_data = pd.DataFrame({'몸무게' : [101]})

result = model.predict(new_data)
print(result[0])

202.5178714537587


In [19]:
#잔차 제곱합
df['잔차'] = df['키'] - model.predict(df)
print("SSE : ", sum(df['잔차']**2))
display(df.head(3))

113.7422663888443


,키,몸무게,잔차
0,150,42,-1.433082
1,160,50,1.640167
2,170,70,-5.676711


In [22]:
#MSE
df['잔차'] = df['키'] - model.predict(df)
print("MSE : ", sum(df['잔차']**2)/len(df))

from sklearn.metrics import mean_squared_error
pred = model.predict(df['몸무게'])
mse = mean_squared_error(df['키'], pred)
print('sklearn MSE : ', mse)

MSE :  5.687113319442215
sklearn MSE :  5.687113319442214


신뢰 구간

In [24]:
#모수에 대한 모델의 신뢰구간
model.conf_int(alpha=0.05)

new_data = pd.DataFrame({'몸무게' : [50]})
pred = model.get_prediction(new_data)
pred.summary_frame(alpha=0.05)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,158.359833,0.794986,156.68963,160.030037,152.820798,163.898869


#다중 선형 회귀 분석

In [27]:
import pandas as pd

data = {
  '매출액': [300, 320, 250, 360, 315, 328, 310, 335,
          326, 280, 290, 300, 315, 328, 310, 335, 300, 400, 500, 680],
  '광고비' : [70, 75, 30, 80, 72, 77, 70, 82, 70, 80,
           68, 90, 72, 77, 70, 82, 40, 20, 75, 80],
  '직원수' : [15, 16, 14, 20, 19, 17, 16, 19, 15, 20,
           14, 5, 16, 17, 16, 14, 30, 40, 10, 50]
}
df = pd.DataFrame(data)
df.head(3)

,매출액,광고비,직원수
0,300,70,15
1,320,75,16
2,250,30,14


In [33]:
from statsmodels.formula.api import ols
model = ols('매출액~광고비+직원수', data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    매출액   R-squared:                       0.585
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     12.00
Date:                Sat, 11 Oct 2025   Prob (F-statistic):           0.000563
Time:                        02:21:20   Log-Likelihood:                -109.93
No. Observations:                  20   AIC:                             225.9
Df Residuals:                      17   BIC:                             228.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     42.6031     78.099      0.546      0.592    -122.171     207.378
광고비            2.2885      0.879      2.604      0.019       0.434       4.143
직원수            7.4981      1.557      4.816      0.000       4.213      10.783
==============================================================================
Omnibus:                       25.432   Durbin-Watson:                   1.344
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.347
Skew:                           2.119   Prob(JB):                     6.37e-10
Kurtosis:                       8.732   Cond. No.                         401.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
#상관계수를 구하시오
df['광고비'].corr(df['매출액'])

np.float64(0.14012507663916066)

In [38]:
from scipy import stats
s,p = stats.pearsonr(df['광고비'], df['매출액'])
print(s, p)

0.14012507663916063 0.5557086162136311


In [47]:
#회귀모델의 r2 계수
model.rsquared

np.float64(0.5853523307724171)

In [48]:
model.params

,0
Intercept,42.603098
광고비,2.288526
직원수,7.498101


In [52]:
model.pvalues['광고비']

np.float64(0.01852464860071739)

#범주형 변수

In [ ]:
import pandas as pd
import statsmodels.api as sm
